In [ ]:
import os
import torch

from diffusion_co_design.wfcrl.schema import ScenarioConfig
from diffusion_co_design.wfcrl.diffusion.generator import (
    Generator,
    projection_constraint,
    train_to_eval,
)
from diffusion_co_design.common import OUTPUT_DIR, get_latest_model

SCENARIO = "wfcrl_10"
scenario = ScenarioConfig.from_file(f"conf/{SCENARIO}.yaml")

pretrain_dir = os.path.join(OUTPUT_DIR, "wfcrl", "diffusion", SCENARIO)
latest_checkpoint = get_latest_model(pretrain_dir, "model")

generator = Generator(
    generator_model_path=latest_checkpoint,
    scenario=scenario,
    batch_size=9,
    guidance_wt=5,
)

In [ ]:
batch = generator.generate_batch(batch_size=9)

In [ ]:
pos = torch.zeros((10, 10, 2)).uniform_(-1, 1)
res = train_to_eval(projection_constraint(scenario)(pos), cfg=scenario)
